# Metabolic Network Reconstruction Tutorial

In the following tutorial you will learn how to manipulate a metabolic model with [cobrapy](https://cobrapy.readthedocs.io/en/latest/) such that you can conform with the recommendations of **Box 2** [in the manuscript on community standards](https://doi.org/10.1101/700112).

From the caption of **Box 2** itself:

> Proposed minimum standardized content for a metabolic network reconstruction. We propose that modelers use this list as a guide to help standardize accessibility, content, and quality; however, more comprehensive documentation and more interpretablee and accessible information can only improve the usability and biological relevance of the shared reconstruction.

Throughout this tutorial we will look at content from the example reconstruction `iPfal19.xml` for _Plasmodium falciparum_ 3D7 provided with the publication, as well as building a minimal example from scratch. Each section will show:

1. Python code needed for the inspection and manipulation of a metabolic reconstruction.
2. Excerpts from the resulting [SBML](http://sbml.org/) that is automatically generated.

In [1]:
import logging

In [2]:
logging.basicConfig(level="INFO")

In [3]:
import cobra
from cobra.io import read_sbml_model

The tutorial was created with the following software versions.

In [4]:
cobra.show_versions()


System Information
OS                    Linux
OS-release 5.0.0-23-generic
Python                3.6.8

Package Versions
cobra                       0.16.0
depinfo                      1.5.1
future                      0.17.1
numpy                       1.17.0
optlang                      1.4.4
pandas                      0.25.0
pip                         19.2.1
python-libsbml-experimental 5.18.0
ruamel.yaml                 0.16.1
setuptools                  41.0.1
six                         1.12.0
swiglpk                     4.65.0
wheel                       0.33.4


## Model

In [5]:
p_falciparum = read_sbml_model("iPfal19.xml")

In [12]:
minimal = cobra.Model("minimal", name="Minimal Demo Model")

Simply saving an empty model with an identifier and name to SBML will generate the following XML element:

```xml
<model metaid="meta_minimal" id="minimal" name="Minimal Demo Model" fbc:strict="true">
```

### Recognized naming convention

# TODO: Recommendation is missing.

### Machine-readable reference to organism and species embedded via MIRIAM annotation

We can find good information about a sequenced species at NCBI. _Plasmodium falciparum_ 3D7 is described [here](https://www.ncbi.nlm.nih.gov/genome/33?genome_assembly_id=369845).

In [8]:
p_falciparum.annotation

{}

In [14]:
minimal.annotation["taxonomy"] = "36329"

This ensures that the reconstruction's taxonomy is annotated. The `annotation` attribute is a Python dictionary whose key-value pairs are automatically converted to MIRIAM compatible URIs.

```xml
    <annotation>
      <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:vCard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
        <rdf:Description rdf:about="#meta_minimal">
          <bqbiol:is>
            <rdf:Bag>
              <rdf:li rdf:resource="https://identifiers.org/taxonomy/36329"/>
            </rdf:Bag>
          </bqbiol:is>
        </rdf:Description>
      </rdf:RDF>
    </annotation>
```

Important elements here are the description which is about the `model` element due to the `metaid` (compare with section [Model](#Model)).

```xml
<rdf:Description rdf:about="#meta_minimal">
```

The next important element is the biological qualifier

```xml
<bqbiol:is>
```

you can find out more about its meaning at [combine](https://co.mbine.org/standards/qualifiers) and [BioModels](https://www.ebi.ac.uk/biomodels-main/annotation).

Finally, there is one MIRIAM compliant annotation that was generated by cobrapy.

```xml
<rdf:li rdf:resource="https://identifiers.org/taxonomy/36329"/>
```

### NCBI reference genome

The genome assembly is described [here](https://www.ncbi.nlm.nih.gov/assembly/GCF_000002765.4/).

# TODO: It seems to me that identifiers.org does not support GCF identifiers? At least `refseq` registry pattern does not allow `GCF`.

In [19]:
minimal.annotation["insdc.gca"] = "GCA_000002765.2"

Which leads to the following new SBML element:

```xml
<rdf:li rdf:resource="https://identifiers.org/insdc.gca/GCA_000002765.2"/>
```

In [16]:
minimal.annotation["genedb"] = "Plasmodium_falciparum_3D7"

This generates the following familiar element:

```xml
<rdf:li rdf:resource="https://identifiers.org/genedb/Plasmodium_falciparum_3D7"/>
```

### Author(s) contact information embedded

# TODO: Format recommendation is missing.

## Metabolite

### Human readable, descriptive name

### Charge

### Chemical formula

### InChI strings

### At least one database identifier from a reliable resource

## Biochemical reaction

### Human readable, descriptive name

### Reaction formula

### Stoichiometric coefficients

### At least one database identifier from a reliable resource

### EC code

## Gene

### Name and/or identifier

### DNA sequence ID

### Protein sequence ID

### Position (including chromosome, if applicable)

### Associated reactions (GPR)

## Save Model

In [10]:
from cobra.io import write_sbml_model

In [20]:
write_sbml_model(minimal, "minimal.xml")